In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.io import arff
import pandas as pd
import numpy as np
import random
#import decimal
#decimal.Decimal(0.1)

In [2]:
data = arff.loadarff('FootPred_noMiss_numified_instancePruned.arff')
df = pd.DataFrame(data[0])
df.head()

,home_player1_X_coord,home_player1_Y_coord,home_gk_diving,home_gk_handling,home_gk_kicking,home_gk_positioning,home_gk_reflexes,home_player2_X_coord,home_player2_Y_coord,home_player2_crossing,...,away_team_chanceCreationShootingClass,away_team_chanceCreationPositioningClass,away_team_defencePressure,away_team_defencePressureClass,away_team_defenceAggression,away_team_defenceAggressionClass,away_team_defenceTeamWidth,away_team_defenceTeamWidthClass,away_team_defenceDefenderLineClass,class
0,1.0,1.0,75.0,72.0,70.0,74.0,77.0,2.0,3.0,73.0,...,1.0,1.0,36.0,1.0,45.0,1.0,49.0,1.0,1.0,b'0'
1,1.0,1.0,83.0,81.0,72.0,80.0,83.0,2.0,3.0,64.0,...,3.0,1.0,50.0,1.0,65.0,1.0,44.0,1.0,1.0,b'0'
2,1.0,1.0,74.0,68.0,73.0,64.0,76.0,2.0,3.0,74.0,...,1.0,1.0,41.0,1.0,39.0,1.0,51.0,1.0,1.0,b'0'
3,1.0,1.0,77.0,81.0,74.0,80.0,79.0,2.0,3.0,74.0,...,1.0,1.0,58.0,1.0,60.0,1.0,63.0,1.0,1.0,b'0'
4,1.0,1.0,77.0,80.0,73.0,78.0,84.0,2.0,3.0,70.0,...,1.0,1.0,47.0,1.0,45.0,1.0,49.0,1.0,1.0,b'0'


In [3]:
X = df.iloc[:, 0:695].astype(int)
y = df.iloc[:, 696].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [4]:
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_prob_pred = clf.predict_proba(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.45115995115995117


In [5]:
for c in range(len(y_pred)):#changing draws to the win of the more probable team
    if y_pred[c]==0:
        if y_prob_pred[c][1] > y_prob_pred[c][2]:
            y_pred[c]=1
        elif y_prob_pred[c][2] > y_prob_pred[c][1]:
            y_pred[c]=2
        else:
            y_pred[c]=random.randint(1, 2) #Houston likes living dangerously
            print("Houston we have a problem: " +str(y_pred[c]))
#making my own accuracy measurment that unifies a draw with either side winning
acc=0
for c in range(len(y_pred)):
    if y_pred[c]==y_test.iloc[c] or (y_test.iloc[c]==0 and y_pred[c]!=0):
        acc=acc+1
acc=acc/len(y_pred)
print(acc)
        

Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 1
Houston we have a problem: 1
Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 2
Houston we have a problem: 2
Houston we have a problem: 2
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 2
Houston we have a problem: 1
Houston we have a problem: 1
Houston we hav

In [6]:
#want to see the distribution of correctness depending on confidence
accu=[]
count=[]
for r in range(100):
    count.append(0)
for c in range(len(y_prob_pred)):
    b=y_pred[c]
    temp=y_prob_pred[c][b]
    count[int(round(temp*100))]=(count[int(round(temp*100))])+1
    if (y_pred[c] == y_test.iloc[c] ):
        accu.append(temp)

In [7]:
for a in range(100):
    if count[a]==0:
        noZeroDivFix=1
    else:
        noZeroDivFix=count[a]
    print(str(float(a/100))+" --> "+str(accu.count(float(a/100)))+ " / " +str(count[a])+" --> " + str(accu.count(float(a/100))/noZeroDivFix*100)+"%")

0.0 --> 0 / 0 --> 0.0%
0.01 --> 0 / 0 --> 0.0%
0.02 --> 0 / 0 --> 0.0%
0.03 --> 0 / 0 --> 0.0%
0.04 --> 0 / 0 --> 0.0%
0.05 --> 0 / 0 --> 0.0%
0.06 --> 0 / 0 --> 0.0%
0.07 --> 0 / 0 --> 0.0%
0.08 --> 0 / 0 --> 0.0%
0.09 --> 0 / 0 --> 0.0%
0.1 --> 0 / 0 --> 0.0%
0.11 --> 0 / 0 --> 0.0%
0.12 --> 0 / 0 --> 0.0%
0.13 --> 0 / 0 --> 0.0%
0.14 --> 0 / 1 --> 0.0%
0.15 --> 0 / 0 --> 0.0%
0.16 --> 0 / 0 --> 0.0%
0.17 --> 0 / 0 --> 0.0%
0.18 --> 0 / 0 --> 0.0%
0.19 --> 1 / 1 --> 100.0%
0.2 --> 0 / 0 --> 0.0%
0.21 --> 0 / 0 --> 0.0%
0.22 --> 0 / 0 --> 0.0%
0.23 --> 0 / 0 --> 0.0%
0.24 --> 0 / 0 --> 0.0%
0.25 --> 0 / 0 --> 0.0%
0.26 --> 1 / 4 --> 25.0%
0.27 --> 6 / 14 --> 42.857142857142854%
0.28 --> 6 / 15 --> 40.0%
0.29 --> 10 / 28 --> 35.714285714285715%
0.3 --> 20 / 45 --> 44.44444444444444%
0.31 --> 23 / 106 --> 21.69811320754717%
0.32 --> 47 / 136 --> 34.55882352941176%
0.33 --> 50 / 180 --> 27.77777777777778%
0.34 --> 59 / 225 --> 26.222222222222225%
0.35 --> 78 / 237 --> 32.91139240506329%


In [8]:
print(0.58*100.0)#floats stored in decimal, little difference when multiplied by 100 makes a shift

57.99999999999999


In [10]:
#test if all is good instance number wise
len(y_prob_pred)
t=0
for h in count:
    t=t+h
len(y_prob_pred)==t

True